In [82]:
import sys
sys.path.append('..')

In [83]:
import numpy

In [84]:
p = numpy.array(
    [
        [11, 12, 13],
        [21, 22, 23],
        [31, 32, 33],
    ]
)
p

array([[11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]])

In [117]:
import lasp.utils
p1 = lasp.utils.circshift(p, center=1-numpy.array([2, 2]))
print(p1)

[[22 23 21]
 [32 33 31]
 [12 13 11]]


In [178]:
def make_bccb(p: numpy.ndarray) -> numpy.ndarray :
    
    dim = numpy.prod(numpy.array(p.shape))
    p_cpy = numpy.copy(p)
    
    p1 = numpy.reshape(p_cpy.T, newshape=dim)
    P = numpy.zeros(shape=(dim, dim))
    P[:, 0] = p1

    for i in range(1, 3):
        for j in range(0, 3*3, 3):
            P[j:j+3, i] = numpy.roll(P[j:j+3, i-1], 1)

    for i in range(1, 3):
        P[:, 3*i: 3*i+3] = numpy.roll(P[:, 3*(i-1): 3*(i-1)+3], (3, 0), (0, 1))


    return P

In [180]:
make_bccb(p1)

array([[22., 12., 32., 21., 11., 31., 23., 13., 33.],
       [32., 22., 12., 31., 21., 11., 33., 23., 13.],
       [12., 32., 22., 11., 31., 21., 13., 33., 23.],
       [23., 13., 33., 22., 12., 32., 21., 11., 31.],
       [33., 23., 13., 32., 22., 12., 31., 21., 11.],
       [13., 33., 23., 12., 32., 22., 11., 31., 21.],
       [21., 11., 31., 23., 13., 33., 22., 12., 32.],
       [31., 21., 11., 33., 23., 13., 32., 22., 12.],
       [11., 31., 21., 13., 33., 23., 12., 32., 22.]])

In [167]:
x = numpy.array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])
numpy.roll(x, (1, 0), (0, 1))
# array([[9, 0, 1, 2, 3],
#        [4, 5, 6, 7, 8]])

array([[5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4]])

In [214]:
def make_bccb(h: numpy.ndarray) -> numpy.ndarray :
    
    size_block = h.shape[0]

    center = numpy.ceil(numpy.array(h.shape) / 2).astype(int)
    shift = 1-center
    h_shift = lasp.utils.circshift(h, shift)

    dim = size_block**2

    col = numpy.reshape(h_shift.T, newshape=dim)
    
    bccb = numpy.zeros(shape=(dim, dim))
    bccb[:, 0] = col

    for i in range(1, h.shape[0]):
        for j in range(0, size_block**2, size_block):
            bccb[j:j+size_block, i] = numpy.roll(bccb[j:j+size_block, i-1], 1)

    for i in range(1, 3):
        bccb[:, size_block*i: size_block*i+size_block] = \
            numpy.roll(
                bccb[:, size_block*(i-1): size_block*(i-1)+size_block], 
                (size_block, 0), 
                (0, 1)
            )


    return bccb

In [217]:
h = numpy.array(
    [
        [11, 12, 13],
        [21, 22, 23],
        [31, 32, 33],
    ]
)

H = make_bccb(h)
H

array([[22., 12., 32., 21., 11., 31., 23., 13., 33.],
       [32., 22., 12., 31., 21., 11., 33., 23., 13.],
       [12., 32., 22., 11., 31., 21., 13., 33., 23.],
       [23., 13., 33., 22., 12., 32., 21., 11., 31.],
       [33., 23., 13., 32., 22., 12., 31., 21., 11.],
       [13., 33., 23., 12., 32., 22., 11., 31., 21.],
       [21., 11., 31., 23., 13., 33., 22., 12., 32.],
       [31., 21., 11., 33., 23., 13., 32., 22., 12.],
       [11., 31., 21., 13., 33., 23., 12., 32., 22.]])

array([[198.  -0.j        , -15. +41.21216129j, -15. +17.87630389j,
         -4.5 +2.59807621j, -15.  +2.64490471j, -15.  -2.64490471j,
         -4.5 -2.59807621j, -15. -17.87630389j, -15. -41.21216129j]])

array([ 1.98000000e+02+0.00000000e+00j, -1.06581410e-14+5.19615242e+01j,
       -1.06581410e-14-5.19615242e+01j,  7.77156117e-16+5.19615242e+00j,
        7.77156117e-16-5.19615242e+00j,  8.31489188e-15+0.00000000e+00j,
       -1.37962120e-15+0.00000000e+00j, -1.62773417e-15+2.22224132e-15j,
       -1.62773417e-15-2.22224132e-15j])